In [1]:
# coding=utf-8
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import random
import jaconv
from datetime import date, datetime, timedelta, time
import csv

pd.options.display.max_columns = None

In [2]:
# 元データの取得加工
def df_create():
    url = "http://www.mlit.go.jp/road/census/h27/"
    df_kasyo = pd.read_csv('kasyo17.csv', encoding='utf-8')
    df_zkntrf = pd.read_csv('zkntrf17.csv', encoding='Shift_JISx0213')
    df_kasyo = df_kasyo[["交通調査基本区間番号",
                         "路線／路線名",
                         "道路種別",
                         "起点側／接続路線等／交通調査基本区間番号",
                         "終点側／接続路線等／交通調査基本区間番号",
                         "交通量調査単位区間番号／調査単位区間番号",
                         "区間延長（ｋｍ）",
                         "昼間１２時間平均旅行速度（時間帯別交通量加重）／上り（ｋｍ／ｈ）",
                         "昼間１２時間平均旅行速度（時間帯別交通量加重）／下り（ｋｍ／ｈ）"]]
    df_kasyo.rename(columns={"交通量調査単位区間番号／調査単位区間番号": "交通量調査単位区間番号"}, inplace=True)
    df_zkntrf = df_zkntrf[["交通量調査単位区間番号",
                           "上り・下りの別",
                           "時間帯別自動車類交通量（台／時）／７時台",
                           "時間帯別自動車類交通量（台／時）／８時台",
                           "時間帯別自動車類交通量（台／時）／９時台",
                           "時間帯別自動車類交通量（台／時）／１０時台",
                           "時間帯別自動車類交通量（台／時）／１１時台",
                           "時間帯別自動車類交通量（台／時）／１２時台",
                           "時間帯別自動車類交通量（台／時）／１３時台",
                           "時間帯別自動車類交通量（台／時）／１４時台",
                           "時間帯別自動車類交通量（台／時）／１５時台",
                           "時間帯別自動車類交通量（台／時）／１６時台",
                           "時間帯別自動車類交通量（台／時）／１７時台",
                           "時間帯別自動車類交通量（台／時）／１８時台",
                           "時間帯別自動車類交通量（台／時）／１９時台",
                           "時間帯別自動車類交通量（台／時）／２０時台",
                           "時間帯別自動車類交通量（台／時）／２１時台",
                           "時間帯別自動車類交通量（台／時）／２２時台",
                           "時間帯別自動車類交通量（台／時）／２３時台",
                           "時間帯別自動車類交通量（台／時）／０時台",
                           "時間帯別自動車類交通量（台／時）／１時台",
                           "時間帯別自動車類交通量（台／時）／２時台",
                           "時間帯別自動車類交通量（台／時）／３時台",
                           "時間帯別自動車類交通量（台／時）／４時台",
                           "時間帯別自動車類交通量（台／時）／５時台",
                           "時間帯別自動車類交通量（台／時）／６時台"]]
    # 欠損値が含まれている列を除外
    df_zkntrf.dropna(inplace=True)
    # 元データには車種別も含まれているため、上り・下りのみで合算
    df_zkntrf = df_zkntrf.groupby(["交通量調査単位区間番号", "上り・下りの別"], as_index=False).sum()
    df = pd.merge(df_kasyo, df_zkntrf, on="交通量調査単位区間番号")
    # 起点・終点が記載されていないものはデータ型をそろえるため補完
    df["起点側／接続路線等／交通調査基本区間番号"] = df["起点側／接続路線等／交通調査基本区間番号"].fillna(0)
    df["終点側／接続路線等／交通調査基本区間番号"] = df["終点側／接続路線等／交通調査基本区間番号"].fillna(0)
    df["起点側／接続路線等／交通調査基本区間番号"] = df["起点側／接続路線等／交通調査基本区間番号"].astype('int64')
    df["終点側／接続路線等／交通調査基本区間番号"] = df["終点側／接続路線等／交通調査基本区間番号"].astype('int64')

    grouped = df.groupby(["交通調査基本区間番号",
                          "路線／路線名",
                          "道路種別",
                          "起点側／接続路線等／交通調査基本区間番号",
                          "終点側／接続路線等／交通調査基本区間番号",
                          "交通量調査単位区間番号",
                          "上り・下りの別",
                          "区間延長（ｋｍ）",
                          "昼間１２時間平均旅行速度（時間帯別交通量加重）／上り（ｋｍ／ｈ）",
                          "昼間１２時間平均旅行速度（時間帯別交通量加重）／下り（ｋｍ／ｈ）"
                          ], as_index=False)

    df = grouped.sum()
    return df

In [3]:
# 重み付きグラフ作成
def network_create(df, bound):
    g = nx.DiGraph()
    for i, row in df.iterrows():
        g.add_node(row["交通調査基本区間番号"])
        if row["上り・下りの別"] == bound:
            # 上りのエッジ
            if row["終点側／接続路線等／交通調査基本区間番号"] != 0:
                g.add_edge(row["終点側／接続路線等／交通調査基本区間番号"],
                           row["交通調査基本区間番号"],
                           weight=row["道路種別"],
                           car_speed=row["昼間１２時間平均旅行速度（時間帯別交通量加重）／上り（ｋｍ／ｈ）"],
                           road_length=row["区間延長（ｋｍ）"])
            if row["起点側／接続路線等／交通調査基本区間番号"] != 0:
                g.add_edge(row["交通調査基本区間番号"],
                           row["起点側／接続路線等／交通調査基本区間番号"],
                           weight=row["道路種別"],
                           car_speed=row["昼間１２時間平均旅行速度（時間帯別交通量加重）／上り（ｋｍ／ｈ）"],
                           road_length=row["区間延長（ｋｍ）"])
        if row["上り・下りの別"] == bound:
            # 下りのエッジ
            if row["終点側／接続路線等／交通調査基本区間番号"] != 0:
                g.add_edge(row["交通調査基本区間番号"],
                           row["終点側／接続路線等／交通調査基本区間番号"],
                           weight=row["道路種別"],
                           car_speed=row["昼間１２時間平均旅行速度（時間帯別交通量加重）／下り（ｋｍ／ｈ）"],
                           road_length=row["区間延長（ｋｍ）"])
            if row["起点側／接続路線等／交通調査基本区間番号"] != 0:
                g.add_edge(row["起点側／接続路線等／交通調査基本区間番号"],
                           row["交通調査基本区間番号"],
                           weight=row["道路種別"],
                           car_speed=row["昼間１２時間平均旅行速度（時間帯別交通量加重）／下り（ｋｍ／ｈ）"],
                           road_length=row["区間延長（ｋｍ）"])

    # node_list = []
    # for component in nx.connected_components(g):
    #     if len(component) > 3:
    #         node_list.extend(component)
    # layout = nx.spring_layout(g, iterations=50)
    # nx.draw_networkx_nodes(g, layout, nodelist=node_list, node_size=10)
    # nx.draw_networkx_edges(g, layout, nodelist=node_list, width=1)
    # nx.draw_networkx_nodes(g, layout, node_size=10)
    # nx.draw_networkx_edges(g, layout, width=1)
    # plt.show()
    return g

In [4]:
def get_start_node(df):
    # 道路種別が一番小さいものを一覧で取得
    start_list = df[df["道路種別"] == min(df["道路種別"])]["交通調査基本区間番号"].values.tolist()
    # ランダムで一つ選択
    start_node = random.choice(start_list)
    return start_node

In [5]:
def move_node(info_df, car_df, current_car_id, network, current_node, bound, hour):
    movable_node_list = []
    loop_node_list = []
    # 隣接ノードの一覧を取得
    for k, v in network[current_node].items():
        loop_node_list.append([k, v['weight']])
    # 重み順に並び替え
    loop_node_list = sorted(loop_node_list, key=lambda x: x[1])
    loop_node_df = pd.DataFrame(loop_node_list)
    loop_weight_range = max(loop_node_df[1])+1
    tmp_df = loop_node_df
    # 重み最大値まで繰り返し
    for i in range(1, loop_weight_range):
        # 対応する重みがあれば処理実行
        if len(loop_node_df[loop_node_df[1] == i]):
            # 同一の重みであればランダムで選択
            for j in range(len(loop_node_df[loop_node_df[1] == i])):
                tmp_data = tmp_df[tmp_df[1] == i].sample()
                index = tmp_data.index.values[0]
                key = tmp_data[0].values[0]
                tmp_df.drop(index=index, inplace=True)

                # 移動先ノードから交通量情報が取得できない場合はスキップ
                if key not in info_df["交通調査基本区間番号"].values:
                    car_df.loc[current_car_id, "finish_reason"] = "交通量情報取得不可" + str(key)
                    continue
                # 一度経由したノードはスキップ
                if key in car_df.loc[current_car_id, "via_node"]:
                    car_df.loc[current_car_id, "finish_reason"] = "既経由ノード" + str(key)
                    continue

                movable_node_list.append(key)
                # 処理時に移動先のノードにいる車の数が移動先ノードの交通量を超えていなければ移動
                if len(car_df[car_df["current_node"] == key]) < \
                        info_df[(info_df["交通調査基本区間番号"] == key) & (info_df["上り・下りの別"] == bound)][
                            "時間帯別自動車類交通量（台／時）／" + jaconv.h2z(str(hour), digit=True, ascii=True) + "時台"].values[0]:
                    # ノード移動
                    car_df.loc[current_car_id, "current_node"] = key
                    # 経由地追加
                    car_df.loc[current_car_id, "via_node"].append(key)
                    # 経由時間追加
                    last_over_time = car_df.loc[current_car_id, "over_time"][-1]
                    edge = network[current_node][key]
                    add_over_time = edge['road_length']/edge['car_speed']
                    over_time = datetime.combine(date.today(), last_over_time) + timedelta(hours=add_over_time)
                    car_df.loc[current_car_id, "over_time"].append(over_time.time())
                    # 最終ノード更新
                    car_df.loc[current_car_id, "end_node"] = key
                    # 最終時間更新
                    car_df.loc[current_car_id, "end_time"] = over_time.time()
                    # 処理終了理由更新
                    car_df.loc[current_car_id, "finish_reason"] = ""

                    break

    # 時間帯をオーバーした場合には終了
    if car_df.loc[current_car_id, "end_time"] > time(hour + 1, 0, 0):
        car_df.loc[current_car_id, "finish_flg"] = True
    # 移動できない場合には終了
    if len(movable_node_list) == 0:
        car_df.loc[current_car_id, "finish_flg"] = True

In [8]:
def exec_every_hour(df, bound):
    # 重み付きグラフの作成
    network = network_create(df, bound)

    # startの設定
    start_node = get_start_node(df)

    print("交通調査基本区間番号:", start_node)
    print("路線／路線名:", df[(df["交通調査基本区間番号"] == start_node) & (df["上り・下りの別"] == bound)]["路線／路線名"].values[0])
    print("隣接ノード:", list(network.neighbors(start_node)))

    for hour in range(24):
        # 交通量データが取得できている時間帯のみ処理実行
        if "時間帯別自動車類交通量（台／時）／" + jaconv.h2z(str(hour), digit=True, ascii=True) + "時台" in df.columns:
            print("時間帯別自動車類交通量（台／時）／" + jaconv.h2z(str(hour), digit=True, ascii=True) + "時台 処理開始")
            # 初期変数設定
            car_num = df[(df["交通調査基本区間番号"] == start_node) & (df["上り・下りの別"] == 1)][
                "時間帯別自動車類交通量（台／時）／" + jaconv.h2z(str(hour), digit=True, ascii=True) + "時台"].values[0]
            car_df = pd.DataFrame()
            car_df["id"] = range(car_num)
            car_df["start_node"] = start_node
            car_df["start_time"] = time(hour, 0, 0)
            car_df["current_node"] = start_node
            car_df["via_node"] = car_df.apply(lambda x: [start_node], axis=1)
            car_df["end_node"] = 0
            car_df["end_time"] = time(hour, 0, 0)
            car_df["finish_flg"] = False
            car_df["finish_reason"] = ""
            for car_id, row in car_df.iterrows():
                minute = int(random.random() * 60)
                second = int(random.random() * 60)
                car_df.loc[car_id, "start_time"] = time(hour, minute, second)
                car_df.loc[car_id, "end_time"] = time(hour, minute, second)
            car_df["over_time"] = car_df["start_time"].apply(lambda x: [x])
            print(car_df)
            print("初期変数設定完了")
            # 突き当りもしくは時間で終了

            
            count = 0
            while len(car_df[car_df["finish_flg"].isin([False])]):
                # print(car_df[car_df["finish_flg"].isin([False])])
                # ノード移動
                print("ノード移動")
                for car_id, row in car_df.iterrows():
                    move_node(df, car_df, car_id, network, row["current_node"], bound, hour)
                if count > 150:
                    break
                count += 1
            print("繰り返し処理完了")
            # car_df.drop(columns="finish_reason", inplace=True)
            # 結果出力
            car_df["over_time"] = car_df["over_time"].apply(lambda x: [n.isoformat() for n in x])
            file_name_bound = "上り"
            if bound == 2:
                file_name_bound = "下り"
            car_df.to_csv(jaconv.h2z(str(hour), digit=True, ascii=True) + "時台_" + file_name_bound + "_output.csv",
                          quoting=csv.QUOTE_ALL,
                          index=False,
                          encoding='Shift_JISx0213')
            print("結果出力完了")

In [9]:
def main():
    # CSVデータから処理用データフレームの作成
    df = df_create()
    df.to_csv("calc_data.csv", encoding='Shift_JISx0213')
    # 上り
    bound = 1
    exec_every_hour(df, bound)

    # 下り
    bound = 2
    exec_every_hour(df, bound)

main()

交通調査基本区間番号: 17110800050
路線／路線名: 北陸自動車道
隣接ノード: [17400080350, 17400250160]
時間帯別自動車類交通量（台／時）／０時台 処理開始
      id   start_node start_time  current_node       via_node  end_node  \
0      0  17110800050   00:51:54   17110800050  [17110800050]         0   
1      1  17110800050   00:35:18   17110800050  [17110800050]         0   
2      2  17110800050   00:13:39   17110800050  [17110800050]         0   
3      3  17110800050   00:53:17   17110800050  [17110800050]         0   
4      4  17110800050   00:57:47   17110800050  [17110800050]         0   
5      5  17110800050   00:11:29   17110800050  [17110800050]         0   
6      6  17110800050   00:59:30   17110800050  [17110800050]         0   
7      7  17110800050   00:04:17   17110800050  [17110800050]         0   
8      8  17110800050   00:52:06   17110800050  [17110800050]         0   
9      9  17110800050   00:02:23   17110800050  [17110800050]         0   
10    10  17110800050   00:02:19   17110800050  [17110800050]         0   
1

繰り返し処理完了
結果出力完了
時間帯別自動車類交通量（台／時）／２時台 処理開始
      id   start_node start_time  current_node       via_node  end_node  \
0      0  17110800050   02:24:55   17110800050  [17110800050]         0   
1      1  17110800050   02:05:34   17110800050  [17110800050]         0   
2      2  17110800050   02:53:43   17110800050  [17110800050]         0   
3      3  17110800050   02:13:02   17110800050  [17110800050]         0   
4      4  17110800050   02:50:39   17110800050  [17110800050]         0   
5      5  17110800050   02:01:53   17110800050  [17110800050]         0   
6      6  17110800050   02:18:32   17110800050  [17110800050]         0   
7      7  17110800050   02:34:53   17110800050  [17110800050]         0   
8      8  17110800050   02:58:37   17110800050  [17110800050]         0   
9      9  17110800050   02:23:20   17110800050  [17110800050]         0   
10    10  17110800050   02:05:24   17110800050  [17110800050]         0   
11    11  17110800050   02:30:07   17110800050  [171108000

繰り返し処理完了
結果出力完了
時間帯別自動車類交通量（台／時）／４時台 処理開始
      id   start_node start_time  current_node       via_node  end_node  \
0      0  17110800050   04:31:23   17110800050  [17110800050]         0   
1      1  17110800050   04:32:12   17110800050  [17110800050]         0   
2      2  17110800050   04:23:35   17110800050  [17110800050]         0   
3      3  17110800050   04:00:37   17110800050  [17110800050]         0   
4      4  17110800050   04:09:26   17110800050  [17110800050]         0   
5      5  17110800050   04:23:52   17110800050  [17110800050]         0   
6      6  17110800050   04:00:58   17110800050  [17110800050]         0   
7      7  17110800050   04:44:43   17110800050  [17110800050]         0   
8      8  17110800050   04:50:59   17110800050  [17110800050]         0   
9      9  17110800050   04:23:13   17110800050  [17110800050]         0   
10    10  17110800050   04:14:31   17110800050  [17110800050]         0   
11    11  17110800050   04:16:03   17110800050  [171108000

繰り返し処理完了
結果出力完了
時間帯別自動車類交通量（台／時）／６時台 処理開始
      id   start_node start_time  current_node       via_node  end_node  \
0      0  17110800050   06:06:33   17110800050  [17110800050]         0   
1      1  17110800050   06:35:58   17110800050  [17110800050]         0   
2      2  17110800050   06:02:46   17110800050  [17110800050]         0   
3      3  17110800050   06:10:26   17110800050  [17110800050]         0   
4      4  17110800050   06:36:58   17110800050  [17110800050]         0   
5      5  17110800050   06:02:55   17110800050  [17110800050]         0   
6      6  17110800050   06:11:55   17110800050  [17110800050]         0   
7      7  17110800050   06:08:28   17110800050  [17110800050]         0   
8      8  17110800050   06:52:33   17110800050  [17110800050]         0   
9      9  17110800050   06:19:21   17110800050  [17110800050]         0   
10    10  17110800050   06:55:47   17110800050  [17110800050]         0   
11    11  17110800050   06:33:40   17110800050  [171108000

繰り返し処理完了
結果出力完了
時間帯別自動車類交通量（台／時）／８時台 処理開始
        id   start_node start_time  current_node       via_node  end_node  \
0        0  17110800050   08:40:31   17110800050  [17110800050]         0   
1        1  17110800050   08:23:00   17110800050  [17110800050]         0   
2        2  17110800050   08:29:57   17110800050  [17110800050]         0   
3        3  17110800050   08:28:34   17110800050  [17110800050]         0   
4        4  17110800050   08:34:36   17110800050  [17110800050]         0   
5        5  17110800050   08:48:36   17110800050  [17110800050]         0   
6        6  17110800050   08:08:07   17110800050  [17110800050]         0   
7        7  17110800050   08:27:45   17110800050  [17110800050]         0   
8        8  17110800050   08:17:46   17110800050  [17110800050]         0   
9        9  17110800050   08:03:12   17110800050  [17110800050]         0   
10      10  17110800050   08:30:14   17110800050  [17110800050]         0   
11      11  17110800050   08:32:02

...

繰り返し処理完了
結果出力完了
時間帯別自動車類交通量（台／時）／１８時台 処理開始
        id   start_node start_time  current_node       via_node  end_node  \
0        0  17110800050   18:44:36   17110800050  [17110800050]         0   
1        1  17110800050   18:40:51   17110800050  [17110800050]         0   
2        2  17110800050   18:36:28   17110800050  [17110800050]         0   
3        3  17110800050   18:17:19   17110800050  [17110800050]         0   
4        4  17110800050   18:16:12   17110800050  [17110800050]         0   
5        5  17110800050   18:26:04   17110800050  [17110800050]         0   
6        6  17110800050   18:11:12   17110800050  [17110800050]         0   
7        7  17110800050   18:40:36   17110800050  [17110800050]         0   
8        8  17110800050   18:57:36   17110800050  [17110800050]         0   
9        9  17110800050   18:40:12   17110800050  [17110800050]         0   
10      10  17110800050   18:02:34   17110800050  [17110800050]         0   
11      11  17110800050   18:21:2

繰り返し処理完了
結果出力完了
時間帯別自動車類交通量（台／時）／１９時台 処理開始
      id   start_node start_time  current_node       via_node  end_node  \
0      0  17110800050   19:42:23   17110800050  [17110800050]         0   
1      1  17110800050   19:30:49   17110800050  [17110800050]         0   
2      2  17110800050   19:03:14   17110800050  [17110800050]         0   
3      3  17110800050   19:17:38   17110800050  [17110800050]         0   
4      4  17110800050   19:20:44   17110800050  [17110800050]         0   
5      5  17110800050   19:34:46   17110800050  [17110800050]         0   
6      6  17110800050   19:38:39   17110800050  [17110800050]         0   
7      7  17110800050   19:01:04   17110800050  [17110800050]         0   
8      8  17110800050   19:41:09   17110800050  [17110800050]         0   
9      9  17110800050   19:03:43   17110800050  [17110800050]         0   
10    10  17110800050   19:09:38   17110800050  [17110800050]         0   
11    11  17110800050   19:34:24   17110800050  [17110800

繰り返し処理完了
結果出力完了
時間帯別自動車類交通量（台／時）／２１時台 処理開始
      id   start_node start_time  current_node       via_node  end_node  \
0      0  17110800050   21:23:45   17110800050  [17110800050]         0   
1      1  17110800050   21:41:50   17110800050  [17110800050]         0   
2      2  17110800050   21:31:21   17110800050  [17110800050]         0   
3      3  17110800050   21:54:15   17110800050  [17110800050]         0   
4      4  17110800050   21:15:25   17110800050  [17110800050]         0   
5      5  17110800050   21:44:29   17110800050  [17110800050]         0   
6      6  17110800050   21:50:02   17110800050  [17110800050]         0   
7      7  17110800050   21:32:13   17110800050  [17110800050]         0   
8      8  17110800050   21:07:46   17110800050  [17110800050]         0   
9      9  17110800050   21:24:53   17110800050  [17110800050]         0   
10    10  17110800050   21:20:29   17110800050  [17110800050]         0   
11    11  17110800050   21:04:44   17110800050  [17110800

繰り返し処理完了
結果出力完了
時間帯別自動車類交通量（台／時）／２３時台 処理開始
      id   start_node start_time  current_node       via_node  end_node  \
0      0  17110800050   23:10:22   17110800050  [17110800050]         0   
1      1  17110800050   23:54:05   17110800050  [17110800050]         0   
2      2  17110800050   23:09:37   17110800050  [17110800050]         0   
3      3  17110800050   23:41:21   17110800050  [17110800050]         0   
4      4  17110800050   23:45:28   17110800050  [17110800050]         0   
5      5  17110800050   23:08:48   17110800050  [17110800050]         0   
6      6  17110800050   23:40:04   17110800050  [17110800050]         0   
7      7  17110800050   23:39:07   17110800050  [17110800050]         0   
8      8  17110800050   23:49:06   17110800050  [17110800050]         0   
9      9  17110800050   23:34:30   17110800050  [17110800050]         0   
10    10  17110800050   23:45:42   17110800050  [17110800050]         0   
11    11  17110800050   23:10:37   17110800050  [17110800

ValueError: hour must be in 0..23